# GPUの利用

## GPU利用のために必要となる設定

### インスタンスタイプ、VMサイズ

GPUが利用できるEC2インスタンスタイプ(Amazon)、VMサイズ(Microsoft Azure)を指定する必要があります。

Amazon EC2 であれば、以下のものが候補になります。

* [p3インスタンス](https://aws.amazon.com/jp/ec2/instance-types/p3/)
* [p2インスタンス](https://aws.amazon.com/jp/ec2/instance-types/p2/)
* [g3インスタンス](https://aws.amazon.com/jp/ec2/instance-types/g3/)

Microsoft Azure VM であれば、以下のものが候補になります。

* [NVシリーズ](https://docs.microsoft.com/ja-jp/azure/virtual-machines/windows/sizes-gpu#nv-series)

### ドライバ、ライブラリなど

GPUを利用するためには、GPUドライバ、ライブラリなどを準備する必要があります。

VCPではGPUドライバなどをVCノードの構成に合わせた形でセットアップしたものを事前に用意してあります。そのため通常のVCノードを利用する場合に`spec`指定を行ったのと同様の手順で、いくつかのパラメータを `spec`に指定するだけでGPUを利用することが出来ます。

GPUを利用しない場合（下図左側）と、GPUを利用する場合（下図右側）のVCノードの構成を以下に示します。

![GPUコンテナ](images/201-001.png)

GPUを利用しない場合との違いを以下に示します。
* OS（マシンイメージ）に nvidia ドライバをインストールしている
* Baseコンテナで[NVIDIA Container Runtime for Docker(nvidia-docker)](https://github.com/NVIDIA/nvidia-docker)を利用できるようにセットアップしている

nvidia ドライバはカーネルモジュールになっているため、VMのカーネルバージョンに強く依存しています。そのためnvidiaドライバはBaseコンテナではなくOS（マシンイメージ）にインストールしています。

## 操作手順

この Notebook では以下の操作を行います。

1. GPU用のVCノードを起動する
1. GPUを利用するアプリケーションコンテナを起動する

# GPU用VCノードを起動する

GPUを利用するために必要となるパラメータを `spec`に指定して、VCノードを起動する。

## アクセストークンの入力

VCP SDKを利用するにはVC Controllerのアクセストークンが必要となります。次のセルを実行すると入力枠が表示されるのでアクセストークンの値を入力してください。

> アクセストークン入力後に Enter キーを押すことで入力が完了します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

## VCP SDKの初期化

VCP SDKの初期化を行います。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK

# VCの管理オブジェクトの作成
vcp = VcpSDK(
    vcc_access_token,  # VCCのアクセストークン
)

上のセルの実行結果がエラーとなり以下のようなメッセージが表示されている場合は、入力されたアクセストークンに誤りがあります。

```
2018-09-XX XX:XX:XX,XXX - ERROR - config vc failed: http_status(403)
2018-09-XX XX:XX:XX,XXX - ERROR - 2018/XX/XX XX:XX:XX UTC: VCPAuthException: xxxxxxx:token lookup is failed: permission denied
```

この場合はアクセストークンの入力からやり直してください。

## VCノードのspecを指定する

GPUを利用するVCノードのspecを指定します。

### specの取得

プロバイダと `flavor`を指定して `spec`オブジェクトを取得します。ここでは以下の値を指定します。

* プロバイダ: `aws`
* flavor: `gpu`

`flavor` に `gpu` を指定することで、GPUが利用できるインスタンスタイプが選択されます。

In [ ]:
spec = vcp.get_spec(
    'aws',    # プロバイダ
    'gpu'  # flavor
)

GPUを利用するための nvidiaドライバや、GPUコンテナを実行するためのランタイム（[nvidia-docker2](https://github.com/NVIDIA/nvidia-docker)）を準備したVMイメージ、Baseコンテナイメージを `spec` の設定に追加します。

In [ ]:
# Baseコンテナイメージの設定
spec.image = 'vcp/gpu:1.6.2-gpusopt'
# 仮想マシンイメージの設定
spec.cloud_image = 'niivcp-gpu-a-21.04.0-r0'

### sshの鍵ファイルを設定する

VCノードにsshでログインするためには事前に公開鍵認証の鍵を登録する必要があります。そのための設定をここで行います。

VCノードに登録する公開鍵認証の**公開鍵**のパスを次のセルで指定してください。

In [ ]:
import os
ssh_public_key = os.path.expanduser('~/.ssh/id_rsa.pub')

指定した公開鍵を `spec` に設定します。

In [ ]:
spec.set_ssh_pubkey(ssh_public_key)

後でVCノードにSSHでログインする際に秘密鍵も必要になるので、ここで設定しておきます。次のセルで**秘密鍵**のパスを指定してください。

In [ ]:
ssh_private_key = os.path.expanduser('~/.ssh/id_rsa')

公開鍵と秘密鍵が正しいペアであることをチェックします。次のセルを実行してエラーにならないことを確認してください。

In [ ]:
!grep -q "$(ssh-keygen -y -f {ssh_private_key})" {ssh_public_key}

### specの設定内容の確認

ここまで `spec` に設定した内容を確認してみます。

In [ ]:
print(spec)

## VCノードの起動

VCノードの起動と、VCノードを操作するためのAnsible設定を行います。

### Unitの作成とVCノードの起動

Unitの作成とVCノードの起動を行います。

まずはUnitGroupを作成します。UnitGroupの名前は `handson201`とします。

In [ ]:
unit_group = vcp.create_ugroup(
    'handson201'                   # UnitGroupの名前
)

VCノードを起動する前のUnitとVCノードの状態を確認しておきます。

In [ ]:
from IPython.display import display

# Unitの一覧を DataFrame で表示する
display(unit_group.df_units())

# VCノードの一覧を DataFrame で表示する
display(unit_group.df_nodes())

Unitの作成とVCノードの起動を行います。

> 処理が完了するまで2分～7分程度かかります。

In [ ]:
# Unitの作成（同時に VCノードが作成される）
unit_group.create_unit(
    'gpu1',         # Unit名
    spec
)

起動したUnit, VCノードの一覧を表示します。

In [ ]:
# Unitの一覧を DataFrame で表示する
display(unit_group.df_units())

# VCノードの一覧を DataFrame で表示する
display(unit_group.df_nodes())

### 疎通確認

起動した VC ノードに SSHでログイン出来ることを確認します。

まず、VCノードのIPアドレスを確認します。

In [ ]:
ip_address = unit_group.find_ip_addresses(node_state='RUNNING').pop()
print(ip_address)

ping による疎通確認を行います。

In [ ]:
# NIIのハンズオン環境ではネットワーク構成の事情により本NotebookからVCノードへのping実行はできません。
# !ping -c 4 {ip_address}

VCノードにSSHでログインする準備としてVCノードのホストキーを ~/.ssh/known_hosts に登録します。

In [ ]:
!touch ~/.ssh/known_hosts
# ~/.ssh/known_hosts から古いホストキーを削除する
!ssh-keygen -R {ip_address}
# ~/.ssh/known_hostsにVCノードを登録する
!ssh-keyscan -H {ip_address} >> ~/.ssh/known_hosts

VCノードにSSHでログインをしてコマンドを実行してみます。

In [ ]:
!ssh root@{ip_address} ls -la

# GPU環境の確認

NVIDIA GPUではGPUデバイスを管理、監視するためのコマンド[nvidia-smi](https://developer.nvidia.com/nvidia-system-management-interface)が提供されています。ここでは、起動したVCノードのGPUに関する状態を取得するために`nvidia-smi`コマンドを実行してみます。

`nvidia-smi` コマンドを利用するとGPU名やドライババージョン、GPUメモリ使用量、GPU使用率、GPUの温度などのGPUに関する情報を取得することができます。`nvidia-smi`コマンドが表示するサマリ情報の例を以下に示します。

```
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.130                Driver Version: 384.130                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   51C    P0    59W / 149W |      0MiB / 11439MiB |     96%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU       PID   Type   Process name                             Usage      |
|=============================================================================|
|  No running processes found                                                 |
+-----------------------------------------------------------------------------+
```

`nvidia-smi`コマンドは[CUDAコンテナ](https://hub.docker.com/r/nvidia/cuda/)の中で実行するので、まずコンテナイメージの取得を行います。

> コンテナイメージの取得に１～３分程度かかります。

In [ ]:
!ssh root@{ip_address} docker pull nvidia/cuda:8.0-cudnn5-runtime-ubuntu16.04

実際にCUDAコンテナから`nvidia-smi` コマンドを実行してGPUに関するサマリ情報が表示させてみます。

> コンテナを実行する際に`docker run`コマンドの引数に`--gpus all`オプションを追加しています。このオプションを指定することで、コンテナ内からGPUを利用するための処理が [nvidia-container-runtime](https://github.com/NVIDIA/nvidia-container-runtime) によって実行されています。

In [ ]:
!ssh root@{ip_address} docker run --gpus all --rm \
    nvidia/cuda:8.0-cudnn5-runtime-ubuntu16.04 nvidia-smi

GPUに関する詳細情報を取得するために `--query`オプションを指定して `nvidia-smi` コマンドを実行してみます。

> `nvidia-smi`コマンドの他のオプションについては [nvidia-smi documentation](http://developer.download.nvidia.com/compute/DCGM/docs/nvidia-smi-367.38.pdf)を参照してください。

In [ ]:
!ssh root@{ip_address} docker run --gpus all --rm \
    nvidia/cuda:8.0-cudnn5-runtime-ubuntu16.04 nvidia-smi --query

# GPU環境の利用

構築したGPU環境を利用して [OpenPose](https://github.com/CMU-Perceptual-Computing-Lab/openpose) を実行してみます。OpenPoseは画像、動画から人の体、手、顔、および足のキーポイントを検出するシステムです。

## 準備

起動時のオプションなどを記述した `docker-compose.yml`ファイルをGPU環境に配置します。

OpenPoseのコンテナイメージを作成するための`Dockerfile`とコンテナを起動する際のオプションなどを記述した `run.sh` をGPU環境に配置します。

In [ ]:
!scp -r openpose/ root@{ip_address}:.

## OpenPoseコンテナのビルド（取得）と実行

[OpenPose](https://github.com/CMU-Perceptual-Computing-Lab/openpose)をアプリケーションコンテナとして実行します。

In [ ]:
!ssh root@{ip_address} 'cat openpose/run.sh'

In [ ]:
#!ssh root@{ip_address} 'cd openpose && docker-compose up -d'
!ssh root@{ip_address} 'cd openpose && sh run.sh'

コンテナが起動されたことを確認します。次のセルを実行して`State`が `Up` と表示されていればコンテナの起動に成功しています。

In [ ]:
!ssh root@{ip_address} 'docker ps'

## OpenPoseの利用

### サンプルの画像を処理する

OpenPoseのサンプル画像を処理してみます。

まず処理前のファイルを格納するディレクトリを作成します。

In [ ]:
result_dir = './openpose_result'
!mkdir -p {result_dir}

処理前のファイルを取得します。

In [ ]:
!ssh root@{ip_address} \
    'cd openpose && docker cp openpose:/root/openpose/examples/media .'
!scp -r root@{ip_address}:openpose/media {result_dir}

処理前の画像を１つ表示させてみます。

In [ ]:
from IPython.display import Image
Image(filename='openpose_result/media/COCO_val2014_000000000192.jpg')

[処理前の画像](./openpose_result/media/)のリンクから他の画像を確認することもできます。

OpenPoseで画像の処理を行います。

In [ ]:
!ssh root@{ip_address} 'docker exec -t openpose \
    build/examples/openpose/openpose.bin --display 0 \
    --image_dir examples/media --write_images /root/result \
    --write_images_format jpg'

処理後のファイルを取得します。

In [ ]:
!scp -r root@{ip_address}:openpose/result {result_dir}

OpenPoseで処理した画像を１枚表示させてみます。

In [ ]:
Image(filename='openpose_result/result/COCO_val2014_000000000192_rendered.jpg')

[処理結果](./openpose_result/result/)のリンクから他の処理結果の画像を確認することもできます。

### サンプルの動画を処理する

[処理前の動画](./openpose_result/media/video.avi)を確認します。

In [ ]:
from IPython.display import HTML

HTML("""<video style="max-width: 100%;" controls>
  <source src="openpose_result/media/video.avi" />
</video>
""")


OpenPoseのサンプル動画を処理してみます。

> 処理には30秒程度かかります。

In [ ]:
!ssh root@{ip_address} 'docker exec -t openpose \
    build/examples/openpose/openpose.bin --display 0 \
    --video /root/openpose/examples/media/video.avi --write_video /root/result/result.avi'

処理結果のファイルを取得します。

In [ ]:
!scp -r root@{ip_address}:openpose/result {result_dir}

処理後の動画を確認します。

In [ ]:
# Notebookに埋め込んで再生する場合に、aviだと再生に失敗するが、mp4に変換すると再生できる。
# mp4に変換
!ffmpeg -i openpose_result/result/result.avi openpose_result/result/result.mp4

In [ ]:
from IPython.display import HTML

HTML("""<video style="width: 100%;" controls>
  <source src="openpose_result/result/result.mp4" />
</video>
""")


上記で見られない場合は、[処理後の動画 (result.avi)](./openpose_result/result/result.avi)をダウンロードして確認します。

### GrafanaでVCノードの利用状況を確認する

GrafanaでVCノードの利用状況を確認してみます。

In [ ]:
vcc_ctr = vcp.vcc_info()['host']
http_host = vcc_ctr.split(':')[0]
grafana_url = "https://{}/grafana/d/vcp/vcp-metrics?refresh=5s".format(http_host)
print(grafana_url)

> ログインするためのユーザ、パスワードは `admin`/`admin` です。

### インターネット上の画像を処理する

インターネット上の画像を処理してみます。

処理対象となる画像の URL のリストを設定してください。

In [ ]:
image_urls = [
# (例)
#    'http://www.example.com/sample.jpg',
]

処理対象の画像ファイルをOpenPoseコンテナに配置します。

In [ ]:
for url in image_urls:
    !ssh root@{ip_address} 'cd openpose/data && curl -O {url}'

OpenPoseの処理を実行します。

In [ ]:
if len(image_urls) > 0:
    !ssh root@{ip_address} 'docker exec -t openpose \
        build/examples/openpose/openpose.bin --display 0 \
        --image_dir /root/data/ --write_images /root/result2'

処理結果のファイルを取得します。

In [ ]:
!scp -r root@{ip_address}:openpose/result2 {result_dir}

[処理結果](./openpose_result/result2)を確認します。

# 後始末

ビルドしたOpenPoseのイメージをプライベートレジストリに格納する場合は、次のセルのコメントを削除して実行してください。

> 完了するまで2分程度かかります。

In [ ]:
# !ssh root@{ip_address} 'cd openpose && docker-compose push'

全てのリソースを削除します。

> 処理が完了するまで1分～2分程度かかります。

In [ ]:
unit_group.cleanup()

ビルドのログがある場合は削除します。

In [ ]:
if 'log_file' in locals():
    !rm -f {log_file[1]}

取得した画像ファイルなどを削除します。

In [ ]:
!rm -rf {result_dir}